# Native APIs

Apart from the OpenAI compatible APIs, the SGLang Runtime also provides its native server APIs. We introduce these following APIs:

- `/generate` (text generation model)
- `/get_model_info`
- `/get_server_info`
- `/health`
- `/health_generate`
- `/flush_cache`
- `/update_weights`
- `/encode`(embedding model)
- `/classify`(reward model)

We mainly use `requests` to test these APIs in the following examples. You can also use `curl`.

## Launch A Server

In [1]:
from sglang.utils import (
    execute_shell_command,
    wait_for_server,
    terminate_process,
    print_highlight,
)

import requests

server_process = execute_shell_command(
    """
python3 -m sglang.launch_server --model-path meta-llama/Llama-3.2-1B-Instruct --port=30010
"""
)

wait_for_server("http://localhost:30010")

[2025-01-15 08:39:08] server_args=ServerArgs(model_path='meta-llama/Llama-3.2-1B-Instruct', tokenizer_path='meta-llama/Llama-3.2-1B-Instruct', tokenizer_mode='auto', load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization_param_path=None, quantization=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-3.2-1B-Instruct', chat_template=None, is_embedding=False, revision=None, skip_tokenizer_init=False, host='127.0.0.1', port=30010, mem_fraction_static=0.88, max_running_requests=None, max_total_tokens=None, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='lpm', schedule_conservativeness=1.0, cpu_offload_gb=0, prefill_only_one_req=False, tp_size=1, stream_interval=1, random_seed=696022137, constrained_json_whitespace_pattern=None, watchdog_timeout=300, download_dir=None, base_gpu_id=0, log_level='info', log_level_http=None, log_requests=False, show_time_cost=False, enable_metrics=False, decode_log_interva

[2025-01-15 08:39:26 TP0] Init torch distributed begin.


[2025-01-15 08:39:26 TP0] Load weight begin. avail mem=78.81 GB


[2025-01-15 08:39:27 TP0] Using model weights format ['*.safetensors']
[2025-01-15 08:39:27 TP0] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.24it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.24it/s]

[2025-01-15 08:39:28 TP0] Load weight end. type=LlamaForCausalLM, dtype=torch.bfloat16, avail mem=76.39 GB
[2025-01-15 08:39:28 TP0] KV Cache is allocated. K size: 33.47 GB, V size: 33.47 GB.
[2025-01-15 08:39:28 TP0] Memory pool end. avail mem=7.45 GB
[2025-01-15 08:39:28 TP0] Capture cuda graph begin. This can take up to several minutes.
  0%|          | 0/23 [00:00<?, ?it/s]

...

100%|██████████| 23/23 [00:04<00:00,  4.85it/s]
[2025-01-15 08:39:33 TP0] Capture cuda graph end. Time elapsed: 4.76 s


[2025-01-15 08:39:33 TP0] max_total_num_tokens=2193171, max_prefill_tokens=16384, max_running_requests=4097, context_len=131072
[2025-01-15 08:39:33] INFO:     Started server process [2120917]
[2025-01-15 08:39:33] INFO:     Waiting for application startup.
[2025-01-15 08:39:33] INFO:     Application startup complete.
[2025-01-15 08:39:33] INFO:     Uvicorn running on http://127.0.0.1:30010 (Press CTRL+C to quit)


[2025-01-15 08:39:33] INFO:     127.0.0.1:52104 - "GET /v1/models HTTP/1.1" 200 OK


[2025-01-15 08:39:34] INFO:     127.0.0.1:52106 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-01-15 08:39:34 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-01-15 08:39:36] INFO:     127.0.0.1:52108 - "POST /generate HTTP/1.1" 200 OK
[2025-01-15 08:39:36] The server is fired up and ready to roll!


## Generate (text generation model)
Generate completions. This is similar to the `/v1/completions` in OpenAI API. Detailed parameters can be found in the [sampling parameters](../references/sampling_params.md).

In [2]:
url = "http://localhost:30010/generate"
data = {"text": "What is the capital of France?"}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-01-15 08:39:38 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 1, cache hit rate: 6.67%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-01-15 08:39:39 TP0] Decode batch. #running-req: 1, #token: 41, token usage: 0.00, gen throughput (token/s): 7.25, #queue-req: 0
[2025-01-15 08:39:39 TP0] Decode batch. #running-req: 1, #token: 81, token usage: 0.00, gen throughput (token/s): 551.47, #queue-req: 0


[2025-01-15 08:39:39 TP0] Decode batch. #running-req: 1, #token: 121, token usage: 0.00, gen throughput (token/s): 549.56, #queue-req: 0
[2025-01-15 08:39:39] INFO:     127.0.0.1:57046 - "POST /generate HTTP/1.1" 200 OK


## Get Model Info

Get the information of the model.

- `model_path`: The path/name of the model.
- `is_generation`: Whether the model is used as generation model or embedding model.
- `tokenizer_path`: The path/name of the tokenizer.

In [3]:
url = "http://localhost:30010/get_model_info"

response = requests.get(url)
response_json = response.json()
print_highlight(response_json)
assert response_json["model_path"] == "meta-llama/Llama-3.2-1B-Instruct"
assert response_json["is_generation"] is True
assert response_json["tokenizer_path"] == "meta-llama/Llama-3.2-1B-Instruct"
assert response_json.keys() == {"model_path", "is_generation", "tokenizer_path"}

[2025-01-15 08:39:39] INFO:     127.0.0.1:57048 - "GET /get_model_info HTTP/1.1" 200 OK


## Get Server Info
Gets the server information including CLI arguments, token limits, and memory pool sizes.
- Note: `get_server_info` merges the following deprecated endpoints:
  - `get_server_args`
  - `get_memory_pool_size` 
  - `get_max_total_num_tokens`

In [4]:
# get_server_info

url = "http://localhost:30010/get_server_info"

response = requests.get(url)
print_highlight(response.text)

[2025-01-15 08:39:39] INFO:     127.0.0.1:57054 - "GET /get_server_info HTTP/1.1" 200 OK


## Health Check
- `/health`: Check the health of the server.
- `/health_generate`: Check the health of the server by generating one token.

In [5]:
url = "http://localhost:30010/health_generate"

response = requests.get(url)
print_highlight(response.text)

[2025-01-15 08:39:39 TP0] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 0, cache hit rate: 6.25%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-01-15 08:39:39] INFO:     127.0.0.1:57062 - "GET /health_generate HTTP/1.1" 200 OK


In [6]:
url = "http://localhost:30010/health"

response = requests.get(url)
print_highlight(response.text)

[2025-01-15 08:39:39] INFO:     127.0.0.1:57076 - "GET /health HTTP/1.1" 200 OK


## Flush Cache

Flush the radix cache. It will be automatically triggered when the model weights are updated by the `/update_weights` API.

In [7]:
# flush cache

url = "http://localhost:30010/flush_cache"

response = requests.post(url)
print_highlight(response.text)

[2025-01-15 08:39:39] INFO:     127.0.0.1:57078 - "POST /flush_cache HTTP/1.1" 200 OK
[2025-01-15 08:39:39 TP0] Cache flushed successfully!


## Update Weights From Disk

Update model weights from disk without restarting the server. Only applicable for models with the same architecture and parameter size.

SGLang support `update_weights_from_disk` API for continuous evaluation during training (save checkpoint to disk and update weights from disk).


In [8]:
# successful update with same architecture and size

url = "http://localhost:30010/update_weights_from_disk"
data = {"model_path": "meta-llama/Llama-3.2-1B"}

response = requests.post(url, json=data)
print_highlight(response.text)
assert response.json()["success"] is True
assert response.json()["message"] == "Succeeded to update model weights."
assert response.json().keys() == {"success", "message"}

[2025-01-15 08:39:39] Start update_weights. Load format=auto
[2025-01-15 08:39:39 TP0] Update engine weights online from disk begin. avail mem=5.16 GB


[2025-01-15 08:39:39 TP0] Using model weights format ['*.safetensors']


[2025-01-15 08:39:40 TP0] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.20it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.20it/s]

[2025-01-15 08:39:40 TP0] Update weights end.
[2025-01-15 08:39:40 TP0] Cache flushed successfully!
[2025-01-15 08:39:40] INFO:     127.0.0.1:57082 - "POST /update_weights_from_disk HTTP/1.1" 200 OK


In [9]:
# failed update with different parameter size or wrong name

url = "http://localhost:30010/update_weights_from_disk"
data = {"model_path": "meta-llama/Llama-3.2-1B-wrong"}

response = requests.post(url, json=data)
response_json = response.json()
print_highlight(response_json)
assert response_json["success"] is False
assert response_json["message"] == (
    "Failed to get weights iterator: "
    "meta-llama/Llama-3.2-1B-wrong"
    " (repository not found)."
)

[2025-01-15 08:39:40] Start update_weights. Load format=auto
[2025-01-15 08:39:40 TP0] Update engine weights online from disk begin. avail mem=5.16 GB


[2025-01-15 08:39:40 TP0] Failed to get weights iterator: meta-llama/Llama-3.2-1B-wrong (repository not found).
[2025-01-15 08:39:40] INFO:     127.0.0.1:57086 - "POST /update_weights_from_disk HTTP/1.1" 400 Bad Request


## Encode (embedding model)

Encode text into embeddings. Note that this API is only available for [embedding models](openai_api_embeddings.html#openai-apis-embedding) and will raise an error for generation models.
Therefore, we launch a new server to server an embedding model.

In [10]:
terminate_process(server_process)

embedding_process = execute_shell_command(
    """
python -m sglang.launch_server --model-path Alibaba-NLP/gte-Qwen2-7B-instruct \
    --port 30020 --host 0.0.0.0 --is-embedding
"""
)

wait_for_server("http://localhost:30020")

[2025-01-15 08:39:55] server_args=ServerArgs(model_path='Alibaba-NLP/gte-Qwen2-7B-instruct', tokenizer_path='Alibaba-NLP/gte-Qwen2-7B-instruct', tokenizer_mode='auto', load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization_param_path=None, quantization=None, context_length=None, device='cuda', served_model_name='Alibaba-NLP/gte-Qwen2-7B-instruct', chat_template=None, is_embedding=True, revision=None, skip_tokenizer_init=False, host='0.0.0.0', port=30020, mem_fraction_static=0.88, max_running_requests=None, max_total_tokens=None, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='lpm', schedule_conservativeness=1.0, cpu_offload_gb=0, prefill_only_one_req=False, tp_size=1, stream_interval=1, random_seed=924401223, constrained_json_whitespace_pattern=None, watchdog_timeout=300, download_dir=None, base_gpu_id=0, log_level='info', log_level_http=None, log_requests=False, show_time_cost=False, enable_metrics=False, decode_log_interva

[2025-01-15 08:40:00] Downcasting torch.float32 to torch.float16.


[2025-01-15 08:40:12 TP0] Downcasting torch.float32 to torch.float16.


[2025-01-15 08:40:12 TP0] Overlap scheduler is disabled for embedding models.
[2025-01-15 08:40:12 TP0] Downcasting torch.float32 to torch.float16.
[2025-01-15 08:40:12 TP0] Init torch distributed begin.


[2025-01-15 08:40:12 TP0] Load weight begin. avail mem=78.81 GB


...

Loading safetensors checkpoint shards: 100% Completed | 7/7 [00:11<00:00,  1.59s/it]
Loading safetensors checkpoint shards: 100% Completed | 7/7 [00:11<00:00,  1.60s/it]

[2025-01-15 08:40:25 TP0] Load weight end. type=Qwen2ForCausalLM, dtype=torch.float16, avail mem=64.40 GB
[2025-01-15 08:40:25 TP0] KV Cache is allocated. K size: 27.47 GB, V size: 27.47 GB.
[2025-01-15 08:40:25 TP0] Memory pool end. avail mem=7.42 GB


[2025-01-15 08:40:26 TP0] max_total_num_tokens=1028801, max_prefill_tokens=16384, max_running_requests=4019, context_len=131072
[2025-01-15 08:40:26] INFO:     Started server process [2121868]
[2025-01-15 08:40:26] INFO:     Waiting for application startup.
[2025-01-15 08:40:26] INFO:     Application startup complete.
[2025-01-15 08:40:26] INFO:     Uvicorn running on http://0.0.0.0:30020 (Press CTRL+C to quit)


[2025-01-15 08:40:26] INFO:     127.0.0.1:52638 - "GET /v1/models HTTP/1.1" 200 OK


[2025-01-15 08:40:27] INFO:     127.0.0.1:52644 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-01-15 08:40:27 TP0] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-01-15 08:40:28] INFO:     127.0.0.1:52654 - "POST /encode HTTP/1.1" 200 OK
[2025-01-15 08:40:28] The server is fired up and ready to roll!


In [11]:
# successful encode for embedding model

url = "http://localhost:30020/encode"
data = {"model": "Alibaba-NLP/gte-Qwen2-7B-instruct", "text": "Once upon a time"}

response = requests.post(url, json=data)
response_json = response.json()
print_highlight(f"Text embedding (first 10): {response_json['embedding'][:10]}")

[2025-01-15 08:40:31 TP0] Prefill batch. #new-seq: 1, #new-token: 4, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-01-15 08:40:31] INFO:     127.0.0.1:52656 - "POST /encode HTTP/1.1" 200 OK


## Classify (reward model)

SGLang Runtime also supports reward models. Here we use a reward model to classify the quality of pairwise generations.

In [12]:
terminate_process(embedding_process)

# Note that SGLang now treats embedding models and reward models as the same type of models.
# This will be updated in the future.

reward_process = execute_shell_command(
    """
python -m sglang.launch_server --model-path Skywork/Skywork-Reward-Llama-3.1-8B-v0.2 --port 30030 --host 0.0.0.0 --is-embedding
"""
)

wait_for_server("http://localhost:30030")

[2025-01-15 08:40:44] server_args=ServerArgs(model_path='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', tokenizer_path='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', tokenizer_mode='auto', load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization_param_path=None, quantization=None, context_length=None, device='cuda', served_model_name='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', chat_template=None, is_embedding=True, revision=None, skip_tokenizer_init=False, host='0.0.0.0', port=30030, mem_fraction_static=0.88, max_running_requests=None, max_total_tokens=None, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='lpm', schedule_conservativeness=1.0, cpu_offload_gb=0, prefill_only_one_req=False, tp_size=1, stream_interval=1, random_seed=617797526, constrained_json_whitespace_pattern=None, watchdog_timeout=300, download_dir=None, base_gpu_id=0, log_level='info', log_level_http=None, log_requests=False, show_time_cost=False, enable_metrics=Fals

[2025-01-15 08:41:01 TP0] Overlap scheduler is disabled for embedding models.
[2025-01-15 08:41:01 TP0] Init torch distributed begin.


[2025-01-15 08:41:01 TP0] Load weight begin. avail mem=78.81 GB


[2025-01-15 08:41:02 TP0] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:00<00:00,  2.50it/s]


...

Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.46it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.60it/s]

[2025-01-15 08:41:05 TP0] Load weight end. type=LlamaForSequenceClassification, dtype=torch.bfloat16, avail mem=64.70 GB
[2025-01-15 08:41:05 TP0] KV Cache is allocated. K size: 27.62 GB, V size: 27.62 GB.
[2025-01-15 08:41:05 TP0] Memory pool end. avail mem=8.44 GB


[2025-01-15 08:41:06 TP0] max_total_num_tokens=452516, max_prefill_tokens=16384, max_running_requests=2049, context_len=131072
[2025-01-15 08:41:06] INFO:     Started server process [2122811]
[2025-01-15 08:41:06] INFO:     Waiting for application startup.
[2025-01-15 08:41:06] INFO:     Application startup complete.
[2025-01-15 08:41:06] INFO:     Uvicorn running on http://0.0.0.0:30030 (Press CTRL+C to quit)


[2025-01-15 08:41:06] INFO:     127.0.0.1:44214 - "GET /v1/models HTTP/1.1" 200 OK


[2025-01-15 08:41:07] INFO:     127.0.0.1:44218 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-01-15 08:41:07 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-01-15 08:41:08] INFO:     127.0.0.1:44232 - "POST /encode HTTP/1.1" 200 OK
[2025-01-15 08:41:08] The server is fired up and ready to roll!


In [13]:
from transformers import AutoTokenizer

PROMPT = (
    "What is the range of the numeric output of a sigmoid node in a neural network?"
)

RESPONSE1 = "The output of a sigmoid node is bounded between -1 and 1."
RESPONSE2 = "The output of a sigmoid node is bounded between 0 and 1."

CONVS = [
    [{"role": "user", "content": PROMPT}, {"role": "assistant", "content": RESPONSE1}],
    [{"role": "user", "content": PROMPT}, {"role": "assistant", "content": RESPONSE2}],
]

tokenizer = AutoTokenizer.from_pretrained("Skywork/Skywork-Reward-Llama-3.1-8B-v0.2")
prompts = tokenizer.apply_chat_template(CONVS, tokenize=False)

url = "http://localhost:30030/classify"
data = {"model": "Skywork/Skywork-Reward-Llama-3.1-8B-v0.2", "text": prompts}

responses = requests.post(url, json=data).json()
for response in responses:
    print_highlight(f"reward: {response['embedding'][0]}")

[2025-01-15 08:41:18 TP0] Prefill batch. #new-seq: 1, #new-token: 68, #cached-token: 1, cache hit rate: 1.32%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-01-15 08:41:18 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 62, cache hit rate: 43.45%, token usage: 0.00, #running-req: 1, #queue-req: 0
[2025-01-15 08:41:18] INFO:     127.0.0.1:37760 - "POST /classify HTTP/1.1" 200 OK


In [14]:
terminate_process(reward_process)

## Skip Tokenizer and Detokenizer

SGLang Runtime also supports skip tokenizer and detokenizer. This is useful in cases like integrating with RLHF workflow.

In [15]:
tokenizer_free_server_process = execute_shell_command(
    """
python3 -m sglang.launch_server --model-path meta-llama/Llama-3.2-1B-Instruct --port=30010 --skip-tokenizer-init
"""
)

wait_for_server("http://localhost:30010")

[2025-01-15 08:41:30] server_args=ServerArgs(model_path='meta-llama/Llama-3.2-1B-Instruct', tokenizer_path='meta-llama/Llama-3.2-1B-Instruct', tokenizer_mode='auto', load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization_param_path=None, quantization=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-3.2-1B-Instruct', chat_template=None, is_embedding=False, revision=None, skip_tokenizer_init=True, host='127.0.0.1', port=30010, mem_fraction_static=0.88, max_running_requests=None, max_total_tokens=None, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='lpm', schedule_conservativeness=1.0, cpu_offload_gb=0, prefill_only_one_req=False, tp_size=1, stream_interval=1, random_seed=837084438, constrained_json_whitespace_pattern=None, watchdog_timeout=300, download_dir=None, base_gpu_id=0, log_level='info', log_level_http=None, log_requests=False, show_time_cost=False, enable_metrics=False, decode_log_interval

[2025-01-15 08:41:48 TP0] Init torch distributed begin.


[2025-01-15 08:41:48 TP0] Load weight begin. avail mem=78.81 GB


[2025-01-15 08:41:49 TP0] Using model weights format ['*.safetensors']
[2025-01-15 08:41:49 TP0] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.28it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.28it/s]

[2025-01-15 08:41:50 TP0] Load weight end. type=LlamaForCausalLM, dtype=torch.bfloat16, avail mem=76.39 GB
[2025-01-15 08:41:50 TP0] KV Cache is allocated. K size: 33.47 GB, V size: 33.47 GB.
[2025-01-15 08:41:50 TP0] Memory pool end. avail mem=7.45 GB
[2025-01-15 08:41:50 TP0] Capture cuda graph begin. This can take up to several minutes.
  0%|          | 0/23 [00:00<?, ?it/s]

...

 96%|█████████▌| 22/23 [00:04<00:00,  6.17it/s]

100%|██████████| 23/23 [00:04<00:00,  5.06it/s]
[2025-01-15 08:41:55 TP0] Capture cuda graph end. Time elapsed: 4.55 s
[2025-01-15 08:41:55 TP0] max_total_num_tokens=2193171, max_prefill_tokens=16384, max_running_requests=4097, context_len=131072
[2025-01-15 08:41:55] INFO:     Started server process [2123752]
[2025-01-15 08:41:55] INFO:     Waiting for application startup.
[2025-01-15 08:41:55] INFO:     Application startup complete.
[2025-01-15 08:41:55] INFO:     Uvicorn running on http://127.0.0.1:30010 (Press CTRL+C to quit)


[2025-01-15 08:41:55] INFO:     127.0.0.1:44802 - "GET /v1/models HTTP/1.1" 200 OK


[2025-01-15 08:41:56] INFO:     127.0.0.1:44804 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-01-15 08:41:56 TP0] Prefill batch. #new-seq: 1, #new-token: 3, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-01-15 08:41:58] INFO:     127.0.0.1:44814 - "POST /generate HTTP/1.1" 200 OK
[2025-01-15 08:41:58] The server is fired up and ready to roll!


In [16]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")

input_text = "What is the capital of France?"

input_tokens = tokenizer.encode(input_text)
print_highlight(f"Input Text: {input_text}")
print_highlight(f"Tokenized Input: {input_tokens}")

response = requests.post(
    "http://localhost:30010/generate",
    json={
        "input_ids": input_tokens,
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 256,
            "stop_token_ids": [tokenizer.eos_token_id],
        },
        "stream": False,
    },
)
output = response.json()
output_tokens = output["token_ids"]

output_text = tokenizer.decode(output_tokens, skip_special_tokens=False)
print_highlight(f"Tokenized Output: {output_tokens}")
print_highlight(f"Decoded Output: {output_text}")
print_highlight(f"Output Text: {output['meta_info']['finish_reason']}")

[2025-01-15 08:42:00 TP0] Prefill batch. #new-seq: 1, #new-token: 8, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-01-15 08:42:00 TP0] Decode batch. #running-req: 1, #token: 41, token usage: 0.00, gen throughput (token/s): 6.89, #queue-req: 0
[2025-01-15 08:42:00 TP0] Decode batch. #running-req: 1, #token: 81, token usage: 0.00, gen throughput (token/s): 564.88, #queue-req: 0
[2025-01-15 08:42:00] INFO:     127.0.0.1:53644 - "POST /generate HTTP/1.1" 200 OK


In [17]:
terminate_process(tokenizer_free_server_process)